In [12]:
import lxml.etree as ET
import os

In [13]:
input_folder = "test_files_input"
output_folder = "test_files_output"
fns = os.listdir(input_folder)

In [14]:
ET.parse(os.path.join(input_folder, fns[0]))

XMLSyntaxError: StartTag: invalid element name, line 1535, column 3 (test_files_input/MEMORY001A.RC0, line 1535)

hmm. so it seems this is not actually valid xml and if I want to deal with it programmatically, I need to treat it as a text file...

In [15]:
def read_file(file):
    with open(file, 'r') as reader:
        # Note: readlines doesn't trim the line endings
        all_data = reader.readlines()
    return all_data

In [16]:
def write_B_file(data, file):
    with open(file, 'w') as writer:
        for line in data:
            # deal with counter at the end
            if "count" in line:
                start_index_nr = line.find(">") + 1
                end_index_nr = line.find("<", 2)
                nr = int(line[start_index_nr:end_index_nr])
                new_nr = nr + 1
                len_new_nr = len(str(new_nr))
                new_count = "".join((4-len_new_nr)*["0"]) + str(new_nr)
                line = line[:start_index_nr] + new_count + line[end_index_nr:]
            
            # write the line
            writer.write(line)

In [17]:
def write_A_file(data, file):
    with open(file, 'w') as writer:
        flag_rec = False
        flag_track = False
        for line in data:
            # the play to overdub switch
            if "<REC>" in line:
                flag_rec = True
            elif "</REC>" in line:
                flag_rec = False
            if flag_rec and "<A>" in line:
                line = line.replace("1", "0")

            # the tempo sync switch
            if "<TRACK" in line:
                flag_track = True
            elif "</TRACK" in line:
                flag_track = False
            if flag_track and "<M>" in line:
                line = line.replace("1", "0")
            
            # deal with counter at the end
            if "count" in line:
                start_index_nr = line.find(">") + 1
                end_index_nr = line.find("<", 2)
                nr = int(line[start_index_nr:end_index_nr])
                new_nr = nr + 1
                len_new_nr = len(str(new_nr))
                new_count = "".join((4-len_new_nr)*["0"]) + str(new_nr)
                line = line[:start_index_nr] + new_count + line[end_index_nr:]

            # write the line
            writer.write(line)

now loop through all the files; always read, then write with appropriate function

In [18]:
for fn in fns:
    data = read_file(os.path.join(input_folder, fn))
    if "A.RC0" in fn:
        write_A_file(data, os.path.join(output_folder, fn))
    elif "B.RC0" in fn:
        write_B_file(data, os.path.join(output_folder, fn))

code seems to be doing what it should but the looper does not recongnise the changes :-/ not sure what I am doing wrong...